<h2 style="font-weight: bold">Titanic Competition</h2>

<h4>This is my first published notebook ever, So yeah it can't be about something other than the Titanic Competition 😄<br><br>I will be doing a simple then advanced EDA, Data Visualization and Pre-Processing. I also will test different models and techniques to improve my score.<br></h4>

* <h5 style="font-weight: 700">Your feedback is very welcome</h5>
* <h5 style="font-weight: 700">If you find this notebook useful, please don't forget to upvote it!</h5>


In [1]:
# Required packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
import os
os.getcwd()

'C:\\Users\\taha_\\OneDrive - ine.inpt.ma\\Coding\\Data Science\\Kaggle\\notebooks\\Titanic'

In [3]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../input/titanic/train.csv'

In [ ]:
train.head()

#  **Exploratory Data Analysis**

In [ ]:
# Getting to know data
print(train.shape)
print(test.shape)

In [ ]:
# summary of numerical variable
train.describe()

In [ ]:
train.describe()

In [ ]:
# summary of categorial variable
train.info()

In [ ]:
test.info()

In [ ]:
# checking null values
print("training data\n",train.isnull().sum())
print("\ntesting data\n",test.isnull().sum())

In [ ]:
# let's clean visualizations :)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# Visualization
count_plt = sns.countplot(train["Survived"])
plt.show(count_plt)

Sex_plt = sns.countplot(x= "Survived",data=train, hue="Sex")
plt.show(Sex_plt)

Embarked_plt = sns.countplot(x="Survived", data=train, hue="Embarked")
plt.show(Embarked_plt)

Pclass_plt = sns.countplot(x="Survived", data=train, hue="Pclass")
plt.show(Pclass_plt)

SibSp_plt = sns.boxplot(x="SibSp", y= "Survived", data=train)
plt.show(SibSp_plt)

Parch_plt = sns.boxplot(x="Parch", y= "Survived", data=train)
plt.show(Parch_plt)

Age_plt = sns.distplot(train["Age"])
plt.show(Age_plt)

In [ ]:
# Correlation heatmap
f, ax = plt.subplots(figsize=(10, 8))
corr = train.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool),linewidths=0.1,annot=True, cmap=sns.diverging_palette(150, 10, as_cmap=True),
            square=True, ax=ax)

# **Pre-Processing**

In [ ]:
# age_mean, fair mean
train['Age'].fillna(train['Age'].mean(), inplace = True)
test['Age'].fillna(train['Age'].mean(), inplace = True)
train['Embarked'].fillna('S', inplace = True)
test['Fare'].fillna(train['Fare'].mean(), inplace = True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# drop cabin column
train.drop(columns=["Cabin", "Name"], axis=1, inplace=True)
test.drop(columns=["Cabin", "Name"], axis=1, inplace=True)

In [ ]:
# Removing insignificant variables

# change categorical to int for train
embarked=pd.get_dummies(train["Embarked"], drop_first=True)
sex=pd.get_dummies(train["Sex"], drop_first=True)

train=pd.concat([train, embarked, sex],axis=1)
train.drop(columns=["Sex", "Embarked", "Ticket"], axis=1, inplace=True)

# change categorical to int for test
embarked=pd.get_dummies(test["Embarked"], drop_first=True)
sex=pd.get_dummies(test["Sex"], drop_first=True)

test=pd.concat([test, embarked, sex],axis=1)
test.drop(columns=["Sex", "Embarked", "Ticket"], axis=1, inplace=True)

In [ ]:
# checking data
print(train.head())
print(test.head())
print(train.corr())

In [ ]:
train.info()

In [ ]:
test.info()

# **Training and Predicting**

In [ ]:
# Classification Methods

# Logistic Regression
train_x= train.drop(columns=["Survived"], axis=1)
train_y= train["Survived"]

test_x= test.copy()

logistic = LogisticRegression(solver='liblinear')
logistic.fit(train_x, train_y)

prediction_y= logistic.predict(test_x)

In [ ]:
print(logistic.score(train_x, train_y))

In [ ]:
# Submission

output= pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": prediction_y})
output.to_csv("Submission.csv", index=False)
print("Done")